In [21]:
import os
import pandas as pd

In [22]:
INPUT_DIR = '../input_data/IFs'

In [23]:
files = [
    f"{INPUT_DIR}/{f}" for f in os.listdir(INPUT_DIR)
    if os.path.isfile(os.path.join(INPUT_DIR, f))
]

In [24]:
files[0]

'../input_data/IFs/06. Poverty Headcount less than $2.15 per Day, Log Normal - Millions.csv'

In [67]:
list(pd.read_csv(files[1], header=5))

['Unnamed: 0',
 'Base',
 'FS_ALB_2030',
 'FS_ALB_2050',
 'FS_SM_2030',
 'FS_SM_2050',
 'FW_ALB_2030',
 'FW_ALB_2050',
 'FW_SM_2030',
 'FW_SM_2050',
 'FWS_ALB_2030',
 'FWS_ALB_2050',
 'FWS_SM_2030',
 'FWS_SM_2050',
 'SI_BS_0_5x',
 'SI_BS_2x',
 'SI_BS_4x',
 'SI_BS_6x',
 'SI_SM_0_5x',
 'SI_SM_2x',
 'SI_SM_4x',
 'SI_SM_6x',
 'WI_BS_0_5x',
 'WI_BS_2x',
 'WI_BS_4x',
 'WI_BS_6x',
 'WI_SM_0_5x',
 'WI_SM_2x',
 'WI_SM_4x',
 'WI_SM_6x',
 'WSI_BS_0_5x',
 'WSI_BS_2x',
 'WSI_BS_4x',
 'WSI_BS_6x',
 'WSI_SM_0_5x',
 'WSI_SM_2x',
 'WSI_SM_4x',
 'WSI_SM_6x',
 'Base.1',
 'FS_ALB_2030.1',
 'FS_ALB_2050.1',
 'FS_SM_2030.1',
 'FS_SM_2050.1',
 'FW_ALB_2030.1',
 'FW_ALB_2050.1',
 'FW_SM_2030.1',
 'FW_SM_2050.1',
 'FWS_ALB_2030.1',
 'FWS_ALB_2050.1',
 'FWS_SM_2030.1',
 'FWS_SM_2050.1',
 'SI_BS_0_5x.1',
 'SI_BS_2x.1',
 'SI_BS_4x.1',
 'SI_BS_6x.1',
 'SI_SM_0_5x.1',
 'SI_SM_2x.1',
 'SI_SM_4x.1',
 'SI_SM_6x.1',
 'WI_BS_0_5x.1',
 'WI_BS_2x.1',
 'WI_BS_4x.1',
 'WI_BS_6x.1',
 'WI_SM_0_5x.1',
 'WI_SM_2x.1',
 'WI_SM_4x.